In [ ]:
%reload_ext autoreload
%autoreload 2

from util.scenario import Scenario
import numpy as np
import matplotlib.pyplot as plt
from pyomo_models.base import BaseModel
from pyomo_models.multi_uavs import MultiUavModel
from pyomo.opt import SolverFactory
from simulate.simulate import Simulation, Schedule, Parameters
from util import constants
from tqdm.notebook import tqdm
from strategy.naive import NaiveStrategy

In [ ]:
sc = Scenario.from_file("scenarios/three_drones_circling.yml")
# sc = Scenario.from_file("scenarios/two_longer_path.yaml")

_, ax = plt.subplots(figsize=(4,4), dpi=70)
sc.plot(ax=ax, draw_distances=False)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')

plt.savefig("out/multi_uav_path.pdf", bbox_inches="tight")
plt.show()

In [ ]:
B_min = 0.1
B_max = 1
B_start = [1, 1, 1]
v = [1,1,1]
r_charge = np.array([0.15, 0.15, 0.15])
r_deplete = np.array([0.3, 0.3, 0.3])
parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'v', 'r_charge', 'r_deplete']}

In [ ]:
model = MultiUavModel(sc, parameters)
model.epsilon = 0.1

solver = SolverFactory("mindtpy")
solution = solver.solve(model, mip_solver='gurobi', nlp_solver='ipopt')

In [ ]:
# naive approach
strat = NaiveStrategy(sc, parameters=parameters)
decisions, waiting_times, charging_times = strat.simulate()

In [ ]:
# find max timestamp
# model
max_time_model = model.execution_time()

# naive approach
times_naive = []
for d in range(sc.N_d):
    schedule = Schedule(decisions[d], charging_times[d], waiting_times[d])
    # hacky way to get parameters object
    params = Simulation.from_base_model(BaseModel(sc, parameters), d).params
    
    sim = Simulation(schedule, params)
    t = sim.simulate()[1][-1]
    times_naive.append(t)

max_time_naive = max(times_naive)
max_time = max(max_time_naive, max_time_model)

In [ ]:
fig, axes = plt.subplots(sc.N_d, 1, figsize=(3, sc.N_d), dpi=80, sharey=True)
fig.subplots_adjust(hspace=.4)
for d in range(sc.N_d):
    schedule = Schedule(decisions[d], charging_times[d], waiting_times[d])
    # hacky way to get parameters object
    params = Simulation.from_base_model(BaseModel(sc, parameters), d).params
    
    sim = Simulation(schedule, params)
    sim.plot_charge(ax=axes[d])
    
    # set xtick
    t = sim.simulate()[1][-1]
    axes[d].set_xticks([t], [f"{t:.2f}s"])
    
    axes[d].set_xlim([-1, max_time+1])
    if d != sc.N_d - 1:
        axes[d].set_xlabel("")

plt.savefig("out/model_vs_naive/naive.pdf", bbox_inches="tight")
plt.savefig("out/model_vs_naive/naive.png", dpi=500, bbox_inches="tight")
plt.show()

In [ ]:
fig, axes = plt.subplots(sc.N_d, 1, figsize=(3, sc.N_d), dpi=80, sharey=True)
fig.subplots_adjust(hspace=.4)
for d in range(sc.N_d):
    sim = Simulation.from_base_model(model, d)
    sim.plot_charge(ax=axes[d])
    
    # set xtick
    t = sim.simulate()[1][-1]
    axes[d].set_xticks([t], [f"{t:.2f}s"])
    
    axes[d].set_xlim([-1, max_time+1])  
    if d != sc.N_d - 1:
        axes[d].set_xlabel("")
        
    
plt.savefig("out/model_vs_naive/model.pdf", bbox_inches="tight")
plt.savefig("out/model_vs_naive/model.png", dpi=500, bbox_inches="tight")
plt.show()